In [1]:
from datetime import datetime

from pytket import Circuit

from qnexus.client import auth, circuits, projects, jobs


In [2]:
auth.login()

🌐 Browser login initiated.


╭────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                        │
│         Confirm that the browser shows the following code and click 'allow device':    │
│                                                                                        │
│                                      vlNi4K                                            │
│                                                                                        │
╰────────────────────────────────────────────────────────────────────────────────────────╯

Browser didn't open automatically? Use this link: https://staging.myqos.com/auth/device/browser?otp=vlNi4Ko4L0am6OqsfNRzJzLL1ysp0DFJg4IZKCoLxO3egOyLb9BSIVLklWEryetM1vnOZkWTQsVwHduQYtaKig
✅ Successfully logged in as vanya.eccles@quantinuum.com using the browser.


In [2]:
project_ref = projects.submit(
    name=f"demo_{str(datetime.now())}", 
    description="a demo project", 
    properties={}
)

In [ ]:
projects.add_property(
    project_ref, 
    name="Parameter_1", 
    property_type="float", 
    description="Parameter for my iterative algorithm", 
    required=False
)

In [3]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

In [4]:
my_circuit_ref = circuits.submit(
    circuit=c, 
    project=project_ref,
    name="My circuit",
    description="A new circuit",
    properties={"Parameter_1": 0.1}
)

In [5]:
# Examine the circuit
my_circuit_ref.get_circuit()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [6]:
compile_job_ref = jobs.submit_compile_job(
    name=f"compile_job_{str(datetime.now())}",
    circuits=my_circuit_ref,
    optimisation_level=1,
    project=project_ref
)

In [7]:
compiled_circuit_refs = jobs.compilation_results(compile_job_ref)

compiled_circuit_refs[0].get_circuit()

In [9]:
execute_ref =jobs.submit_execute_job(
    name="My Execute Job",
    circuits=compiled_circuit_refs,
    project=project_ref,
    n_shots=[10]
)

In [10]:
# TODO check on job status

JobRef(id=UUID('56229a62-2913-4432-a383-0168a3eab8ca'), annotations=Annotations(name='My Execute Job', description=None, properties={}), job_type=<JobType.Execute: 'PROCESS'>, last_status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, last_message='', project=ProjectRef(id=UUID('28851f04-6476-4041-a29c-141f7a1d623e'), annotations=Annotations(name='demo_2023-12-08 11:58:37.543074', description='a demo project', properties={'Test Prop': 1})))

In [11]:
my_results = jobs.execution_results(execute_ref)

In [14]:
my_results[0].get_result()

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[192]
 [  0]
 [  0]
 ...
 [  0]
 [192]
 [192]],state=None,unitary=None,density_matrix=None)